# Import packages and database

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
import biogeme.segmentation as seg
from biogeme.expressions import Beta, Variable, Derive
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
from collections import namedtuple

data_file ='https://raw.githubusercontent.com/GustavePellier/MMOB/main/lpmc19.dat'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC

database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

We calculate the total public transport duration and the total driving cost

In [2]:
LPMC["dur_pt"]= LPMC["dur_pt_access"] + LPMC["dur_pt_rail"] + LPMC["dur_pt_bus"] + LPMC["dur_pt_int"] 
LPMC["cost_drive"] = LPMC["cost_driving_ccharge"] + LPMC["cost_driving_fuel"]

Some variables are created with the columns that seem to be useful

In [3]:
travel_mode=Variable('travel_mode')

dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')

pt_interchanges=Variable('pt_interchanges')

cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
cost_drive=Variable('cost_drive')
cost_pt=Variable('cost_transit')

time_walk=Variable('dur_walking')
time_cycle=Variable('dur_cycling')
time_pt=Variable('dur_pt')
time_drive=Variable('dur_driving')

female=Variable('female')
age=Variable('age')

There are 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [4]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

Utility functions creation

In [5]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

availability of each mode, all available here

In [6]:
av = {1: 1, 2: 1, 3: 1, 4:1}

The estimation results (parameter values, t-tests or p-values, null and final log likelihoods)

In [7]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.878619,0.107672,-36.022418,0.0
asc_drive,-1.295191,0.080505,-16.088354,0.0
asc_pt,-0.503354,0.054123,-9.300119,0.0
beta_cost,-0.193629,0.013958,-13.871889,0.0
beta_time,-5.495527,0.208596,-26.345342,0.0


In [9]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

In [10]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1

We choose to do a specification for the time attribute, the cost stays generic in this model. Therefore we obtain a beta_time parameter for each transportation mode. 

In [11]:
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)

The utility functions are rewritten with the new parameters :

In [12]:
v_walk_model1= beta_time_walk * time_walk  
v_cycle_model1= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1= asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1= asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

The results are computed (parameter values, t-tests or p-values, null and final log likelihoods)

In [13]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model_1'
all_results['Model1'] = biogeme_model1.estimate()
results_specific_time = biogeme_model1.estimate()
results_specific_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.847427,0.202567,-23.929963,0.0
asc_drive,-1.969868,0.137928,-14.281882,0.0
asc_pt,-2.337230,0.139505,-16.753691,0.0
beta_cost,-0.182006,0.016086,-11.314629,0.0
beta_time_cycle,-5.311244,0.461876,-11.499292,0.0
beta_time_drive,-6.473579,0.378946,-17.083101,0.0
beta_time_pt,-3.538116,0.250631,-14.116823,0.0
beta_time_walk,-8.430537,0.418984,-20.121406,0.0


In [14]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

In [ ]:
biogeme_model1.calculateNullLoglikelihood(av)

We compare the 2 models with the likelihood ratio test because model 0 is a reduction of model 1.  

In [ ]:
results_specific_time.likelihood_ratio_test(results_generic, 0.05)

# Model 2

Using Model 1 as the base model, we choose to study how gender interact with the
ASCs and the alternative attributes. 

Lets start by segmenting the population by gender

In [15]:
gender_segmentation = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

### ASC segmentation

In [16]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
segmented_asc_cycle = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt = Beta('asc_pt', 0, None, None, 0)
segmented_asc_pt = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive = Beta('asc_drive', 0, None, None, 0)
segmented_asc_drive = seg.Segmentation(asc_drive,[gender_segmentation]).segmented_beta()

We redefine the value function of the 4 alternatives, introducing the segmented ASC and the segmented cost attribute

In [17]:
v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

We can compute the result of the new model

In [18]:
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model2'] = biogeme_model2.estimate()
results_segmented_gender = biogeme_model2.estimate()
results_segmented_gender.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.455310,0.253367,-21.531290,0.000000e+00
asc_cycle_male,1.051753,0.208782,5.037565,4.714910e-07
asc_drive,-1.893898,0.144318,-13.123109,0.000000e+00
asc_drive_male,-0.198075,0.095256,-2.079398,3.758075e-02
asc_pt,-2.208753,0.146387,-15.088471,0.000000e+00
asc_pt_male,-0.333281,0.099556,-3.347685,8.148970e-04
beta_cost,-0.182830,0.016153,-11.318948,0.000000e+00
beta_time_cycle,-5.579507,0.473020,-11.795505,0.000000e+00
beta_time_drive,-6.486638,0.379981,-17.070947,0.000000e+00
beta_time_pt,-3.526181,0.251365,-14.028116,0.000000e+00


In [19]:
biogeme_model2.calculateNullLoglikelihood(av)

-6931.471805599917

### Cost segmentation

And its interaction on the cost attribute in the car and public transport alternative.

In [20]:
beta_cost = Beta('beta_cost', 0, None, None, 0)
segmented_beta_cost = seg.Segmentation(beta_cost, [gender_segmentation]).segmented_beta()

In [21]:
v_walk_model2_2= beta_time_walk * time_walk  
v_cycle_model2_2= asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2_2= asc_pt + beta_time_pt * time_pt + segmented_beta_cost * cost_pt
v_drive_model2_2= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

We can compute the result of the new model

In [22]:
V_model2_2 = {1: v_walk_model2_2 , 2: v_cycle_model2_2, 3: v_pt_model2_2, 4: v_drive_model2_2}
logprob_model2_2 = models.loglogit(V_model2_2, av, travel_mode)
biogeme_model2_2 = bio.BIOGEME(database, logprob_model2_2)
biogeme_model2_2.modelName = 'Model_2_2'
all_results['Model2_2'] = biogeme_model2_2.estimate()
results_segmented_gender_2 = biogeme_model2_2.estimate()
results_segmented_gender_2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.844043,0.202230,-23.953172,0.000000e+00
asc_drive,-1.972262,0.137773,-14.315313,0.000000e+00
asc_pt,-2.343553,0.139347,-16.818115,0.000000e+00
beta_cost,-0.151590,0.018751,-8.084407,6.661338e-16
beta_cost_male,-0.065408,0.031847,-2.053798,3.999522e-02
beta_time_cycle,-5.351084,0.459476,-11.646053,0.000000e+00
beta_time_drive,-6.485059,0.380531,-17.042141,0.000000e+00
beta_time_pt,-3.538286,0.250773,-14.109506,0.000000e+00
beta_time_walk,-8.437822,0.418576,-20.158413,0.000000e+00


In [23]:
biogeme_model2_2.calculateNullLoglikelihood(av)

-6931.471805599917

### Compiling all results from the different models

In [27]:
res.compile_estimation_results(all_results)

AttributeError: module 'biogeme.results' has no attribute 'compile_estimation_results'

### Likelihood ratio test to check if model_2 and model_2_2 are equivalent or not

In [28]:
results_segmented_gender.likelihood_ratio_test(results_segmented_gender_2, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=55.92763011476018, threshold=5.991464547107979)

Model_2 is preferred because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2

### Comparison with model 1

In [29]:
results_segmented_gender.likelihood_ratio_test(results_specific_time, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=60.51260183111481, threshold=7.814727903251179)

Model_2 is preferred to Model_1 because they are not equivalent. The BIC and AIC are smaller and the final likelihood is bigger for model 2.

# Model 3 - Thomas

In [30]:
# Repartir du model segmenté, Modèle 2

In [31]:
#GENDER segmentation
gender_segmentation_3bis = seg.DiscreteSegmentationTuple(variable=female, mapping={1: 'female', 0: 'male'})

In [32]:
asc_cycle_3bis = Beta('asc_cycle_3bis', 0, None, None, 0)
segmented_asc_cycle_3bis = seg.Segmentation(asc_cycle,[gender_segmentation]).segmented_beta()

asc_pt_3bis = Beta('asc_pt_3bis', 0, None, None, 0)
segmented_asc_pt_3bis = seg.Segmentation(asc_pt,[gender_segmentation]).segmented_beta()

asc_drive_3bis = Beta('asc_drive_3bis', 0, None, None, 0)
segmented_asc_drive_3bis = seg.Segmentation(asc_drive_3bis,[gender_segmentation_3bis]).segmented_beta()

In [33]:
beta_time_walk_3bis = Beta('beta_time_walk_3bis', 0, None, None, 0)
beta_time_cycle_3bis = Beta('beta_time_cycle_3bis', 0, None, None, 0)

beta_time_pt_3bis = Beta('beta_time_pt_3bis', 0, None, None, 0)
beta_time_pt_3bis_squarred = Beta('beta_time_pt_3bis_squarred', 0, None, None, 0)

beta_time_drive_3bis = Beta('beta_time_drive_3bis', 0, None, None, 0)
beta_cost_3bis = Beta('beta_cost_3bis', 0, None, None, 0)


v_walk_model_3bis= beta_time_walk_3bis * time_walk  

v_cycle_model_3bis= segmented_asc_cycle_3bis + beta_time_cycle_3bis * time_cycle 

v_pt_model_3bis= segmented_asc_pt_3bis + beta_time_pt_3bis * time_pt + beta_time_pt_3bis_squarred *time_pt*time_pt + beta_cost_3bis * cost_pt

v_drive_model_3bis= segmented_asc_drive_3bis + beta_time_drive_3bis * time_drive + beta_cost_3bis * cost_drive

In [34]:
V_model_3bis = {1: v_walk_model_3bis , 2: v_cycle_model_3bis, 3: v_pt_model_3bis, 4: v_drive_model_3bis}
logprob_model_3bis = models.loglogit(V_model_3bis, av, travel_mode)
biogeme_model_3bis = bio.BIOGEME(database, logprob_model_3bis)
biogeme_model_3bis.modelName = 'Model_3bis'
all_results['Model_3bis'] = biogeme_model_3bis.estimate()
results_segmented_gender_3bis = biogeme_model_3bis.estimate()
results_segmented_gender_3bis.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-5.373587,0.250692,-21.434983,0.000000e+00
asc_cycle_male,1.052596,0.208480,5.048916,4.443231e-07
asc_drive_3bis,-1.801700,0.145465,-12.385841,0.000000e+00
asc_drive_3bis_male,-0.197945,0.094442,-2.095949,3.608673e-02
asc_pt,-2.490613,0.160121,-15.554547,0.000000e+00
asc_pt_male,-0.339668,0.101135,-3.358543,7.835466e-04
beta_cost_3bis,-0.180471,0.016392,-11.009875,0.000000e+00
beta_time_cycle_3bis,-5.572447,0.458287,-12.159284,0.000000e+00
beta_time_drive_3bis,-6.562182,0.388230,-16.902819,0.000000e+00
beta_time_pt_3bis,-1.979999,0.470362,-4.209518,2.559159e-05


In [35]:
#COST segmentation

In [37]:
beta_cost_33 = Beta('beta_cost_33', 0, None, None, 0)
segmented_beta_cost_33 = seg.Segmentation(beta_cost_33, [gender_segmentation_3bis]).segmented_beta()

In [38]:
beta_time_pt_33_squarred = Beta('beta_time_pt_33_squarred', 0, None, None, 0)

v_walk_model_33= beta_time_walk * time_walk  

v_cycle_model_33= asc_cycle + beta_time_cycle * time_cycle 

v_pt_model_33= asc_pt + beta_time_pt * time_pt + beta_time_pt_33_squarred * time_pt * time_pt + segmented_beta_cost * cost_pt

v_drive_model_33= asc_drive + beta_time_drive * time_drive + segmented_beta_cost * cost_drive

In [41]:
V_model_33 = {1: v_walk_model_33 , 2: v_cycle_model_33, 3: v_pt_model_33, 4: v_drive_model_33}
logprob_model_33 = models.loglogit(V_model_33, av, travel_mode)
biogeme_model_33 = bio.BIOGEME(database, logprob_model_33)
biogeme_model_33.modelName = 'Model__33'
all_results['Model_33'] = biogeme_model_33.estimate()
results_segmented_gender_33 = biogeme_model_33.estimate()
results_segmented_gender_33.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.761814,0.199493,-23.869533,0.000000e+00
asc_drive,-1.881604,0.139072,-13.529732,0.000000e+00
asc_pt,-2.623117,0.154054,-17.027293,0.000000e+00
beta_cost,-0.149045,0.018924,-7.876005,3.330669e-15
beta_cost_male,-0.065958,0.032232,-2.046391,4.071788e-02
beta_time_cycle,-5.345030,0.445769,-11.990577,0.000000e+00
beta_time_drive,-6.554809,0.388246,-16.883113,0.000000e+00
beta_time_pt,-2.014886,0.470745,-4.280209,1.867181e-05
beta_time_pt_33_squarred,-1.218155,0.352161,-3.459081,5.420228e-04
beta_time_walk,-8.288463,0.418764,-19.792672,0.000000e+00


### Comparison model_3bis and model_33

In [45]:


results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender_33, 0.05)


LRTuple(message='H0 can be rejected at level 5.0%', statistic=56.376802109316486, threshold=5.991464547107979)

In [46]:
print(results_segmented_gender_3bis.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4236.589
Final log likelihood:	-4236.589
Likelihood ratio test for the init. model:	9.05331e-07
Rho-square for the init. model:	1.07e-10
Rho-square-bar for the init. model:	-0.00283
Akaike Information Criterion:	8497.178
Bayesian Information Criterion:	8575.385
Final gradient norm:	2.4392E-02
Nbr of threads:	16



In [47]:
print(results_segmented_gender_33.printGeneralStatistics())

Number of estimated parameters:	10
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4264.778
Final log likelihood:	-4264.778
Likelihood ratio test for the init. model:	1.892004e-07
Rho-square for the init. model:	2.22e-11
Rho-square-bar for the init. model:	-0.00234
Akaike Information Criterion:	8549.555
Bayesian Information Criterion:	8614.727
Final gradient norm:	2.1087E-02
Nbr of threads:	16



### Preferred model is model_3bis

The model_3bis is preferred to model_33 as its log likelihood is greater and the AIC and BIC are smaller as well for this model.

### Comparison model_3bis and model_2

In [48]:
results_segmented_gender_3bis.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=17.30206001629267, threshold=3.841458820694124)

In [49]:
print(results_segmented_gender.printGeneralStatistics())

Number of estimated parameters:	11
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4245.24
Final log likelihood:	-4245.24
Likelihood ratio test for the init. model:	7.329218e-07
Rho-square for the init. model:	8.63e-11
Rho-square-bar for the init. model:	-0.00259
Akaike Information Criterion:	8512.48
Bayesian Information Criterion:	8584.17
Final gradient norm:	2.2127E-02
Nbr of threads:	16



We notice that the Final loglikelyhood as well as AIC & BIC are lower for the model_3bis than for model_2.
As those two model are not equivalent at the 5% significance level, we can conclude that the model 3 bis is preferred

### Version ancienne du power series 

In [ ]:
# Power Series transformation on 'pt_dur'.

asc_cycle_3 = Beta('asc_cycle_3', 0, None, None, 0)

asc_pt_3 = Beta('asc_pt_3', 0, None, None, 0)

asc_drive_3 = Beta('asc_drive_3', 0, None, None, 0)


beta_time_walk_3 = Beta('beta_time_walk_3', 0, None, None, 0)
beta_time_cycle_3 = Beta('beta_time_cycle_3', 0, None, None, 0)

beta_time_pt_3 = Beta('beta_time_pt_3', 0, None, None, 0)
beta_time_pt_3_squarred = Beta('beta_time_pt_3_squarred', 0, None, None, 0)

beta_time_drive_3 = Beta('beta_time_drive_3', 0, None, None, 0)
beta_cost_3 = Beta('beta_cost_3', 0, None, None, 0)



v_walk_model_3= beta_time_walk_3 * time_walk 

v_cycle_model_3= asc_cycle_3 + beta_time_cycle_3 * time_cycle 

v_pt_model_3= asc_pt_3 + beta_time_pt_3 * time_pt +beta_time_pt_3_squarred * time_pt * time_pt + beta_cost * cost_pt

v_drive_model_3= asc_drive_3 + beta_time_drive_3 * time_drive + beta_cost_3 * cost_drive

In [ ]:
V_model_3 = {1: v_walk_model_3 , 2: v_cycle_model_3, 3: v_pt_model_3, 4: v_drive_model_3}

logprob_model_3 = models.loglogit(V_model_3, av, travel_mode)

biogeme_model_3 = bio.BIOGEME(database, logprob_model_3)

biogeme_model_3.modelName = 'Model_3'

all_results['Model_3'] = biogeme_model_3.estimate()

results_power_3 = biogeme_model_3.estimate()

results_power_3.getEstimatedParameters()

In [ ]:
#Calculate Null Log likelihood

biogeme_model_3.calculateNullLoglikelihood(av)

In [25]:
#Comparison to model 1

results_power_3.likelihood_ratio_test(results_specific_time, 0.05)




NameError: name 'results_power_3' is not defined

The model 3 is not preferred compare to model 1.

-Is it due to the non linear transformation that is not appropriate ? Should we use boxplot instead ? --> Yes we should 

-Is it stated 'non-linear transfronation on one of the variables' so I just took public transport time, should I apply to the other time ? --> I think but I will ask

-Should the non linear transformation be applied to an other time variable like car ? Yes I think but I will ask

-Is the loglikelyhood ratio test still appropirate ? --> Maybe we should use a cox-text instead 

Remarks: I think that the non-linear transformation is not done on beta but rather on variables such as time, cost, etc.!


# Model 3 - Salomé

In [50]:

lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0) #vu dans le modèle 01-logit_airline_solution
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)


v_walk_model2= beta_time_walk * time_walk  
v_cycle_model2= segmented_asc_cycle + beta_time_cycle * time_cycle 
v_pt_model2= segmented_asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model2= segmented_asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive


v_walk_model3 = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model3  = segmented_asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model3  = segmented_asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt 
v_drive_model3  = segmented_asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive 

In [51]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.217346,0.303873,-10.587806,0.000000e+00
asc_cycle_male,1.052586,0.209997,5.012384,5.375985e-07
asc_drive,0.069734,0.217639,0.320411,7.486571e-01
asc_drive_male,-0.226137,0.101489,-2.228201,2.586714e-02
asc_pt,1.845428,0.170787,10.805457,0.000000e+00
asc_pt_male,-0.375254,0.107314,-3.496770,4.709273e-04
beta_cost,-0.174895,0.015669,-11.161983,0.000000e+00
beta_elapsed_cycle,-3.317732,0.268365,-12.362776,0.000000e+00
beta_elapsed_time_drive,-3.300818,0.265774,-12.419663,0.000000e+00
beta_elapsed_time_pt,-2.637573,0.182642,-14.441229,0.000000e+00


In [52]:
res.compileEstimationResults(all_results)

(                                              Model0           Model1  \
 Number of estimated parameters                     5                8   
 Sample size                                     5000             5000   
 Final log likelihood                    -4587.818071     -4275.496547   
 Akaike Information Criterion             9185.636142      8566.993094   
 Bayesian Information Criterion           9218.222108      8619.130639   
 asc_cycle (t-test)                      -3.88  (-36)   -4.85  (-23.9)   
 asc_drive (t-test)                     -1.3  (-16.1)   -1.97  (-14.3)   
 asc_pt (t-test)                       -0.503  (-9.3)   -2.34  (-16.8)   
 beta_cost (t-test)                   -0.194  (-13.9)  -0.182  (-11.3)   
 beta_time (t-test)                     -5.5  (-26.3)                    
 beta_time_cycle (t-test)                               -5.31  (-11.5)   
 beta_time_drive (t-test)                               -6.47  (-17.1)   
 beta_time_pt (t-test)                

In [53]:
print("Null Loglikelihood : ")
biogeme_model3.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

To compare Model 2 and Model 3 I think we should use a Cox test as Model 3 is not a a linear restriction of Model 2

In [55]:
results_model3.likelihood_ratio_test(results_segmented_gender, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=175.9877377561279, threshold=3.841458820694124)

In [56]:
print(results_model3.printGeneralStatistics())

Number of estimated parameters:	12
Sample size:	5000
Excluded observations:	0
Init log likelihood:	-4157.246
Final log likelihood:	-4157.246
Likelihood ratio test for the init. model:	-0
Rho-square for the init. model:	0
Rho-square-bar for the init. model:	-0.00289
Akaike Information Criterion:	8338.493
Bayesian Information Criterion:	8416.699
Final gradient norm:	1.9078E-02
Nbr of threads:	16



The coxbox mdel is preferred as it has a lower finalloglikelyhood and BIC & AIC are smaller as well

In [58]:
help( biogeme_boxcox)


NameError: name 'biogeme_boxcox' is not defined